In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import re
import warnings
warnings.filterwarnings('ignore')

# importing the data form csvs

In [14]:
ag_all_bh = pd.read_csv('ag_all_bh.csv')
ag_col_bh = pd.read_csv('ag_col_bh.csv')
ag_merge_bh = pd.read_csv('ag_merge_bh.csv')
ag_col = pd.read_csv('ag_col.csv')
ag_merge = pd.read_csv('ag_merge.csv')


In [15]:
ag_col_bh

,time_x,collision_type,idm,mm,id1,m1,id2,m2,r_x,typem,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
0,11.565179,single-single,1680002,33.8986,1680001,33.7704,470820,0.129842,0.038822,4,...,11.885657,0.191185,0,1680002,0.000000,17.89330,16.09650,0.0,1.00000,data\N16rv0.5rg2.0Z0.002
1,7.978654,single-single,1680061,133.8740,1680060,179.9130,299190,0.394966,0.013368,4,...,8.287822,0.137582,1,1680061,0.000000,33.00390,29.70350,0.0,0.00000,data\N16rv0.5rg2.0Z0.002
2,3.710148,single-single,1680133,110.5980,1680132,110.4200,261872,0.179723,0.025530,4,...,4.003505,0.144856,1,1680133,0.000000,35.19190,31.67270,0.0,0.00000,data\N16rv0.5rg2.0Z0.002
3,4.100341,single-single,1680220,41.3707,1680219,41.2697,328483,0.105145,0.052498,4,...,4.334216,0.069415,0,1680220,0.341818,34.12200,30.70980,0.0,1.00000,data\N16rv0.5rg2.0Z0.002
4,3.849036,single-single,1680292,52.0011,1680291,51.8734,137163,0.133144,0.041029,4,...,4.083307,0.034394,0,1680292,0.169725,40.61420,36.55280,0.0,1.00000,data\N16rv0.5rg2.0Z0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25558,7.467068,single-single,867397,27.4788,27397,27.2059,34148,0.274166,0.809076,4,...,7.467073,0.809076,0,867397,1.377720,29.83110,12.91490,0.0,196.38100,data\N8.0rv4.0rg8.0Z0.002
25559,8.195550,single-single,935010,21.1569,95010,21.0637,260784,0.094146,1.787344,4,...,8.195555,1.787344,0,935010,0.376952,29.08860,11.89070,0.0,89.37041,data\N8.0rv4.0rg8.0Z0.002
25560,8.195550,single-single,1021125,23.4548,239877,23.3603,181125,0.095204,2.219268,4,...,8.195555,2.219268,0,1021125,0.000000,28.60760,11.48710,0.0,149.45210,data\N8.0rv4.0rg8.0Z0.002
25561,6.479118,single-single,901838,26.2744,61838,26.0639,69692,0.210457,0.995484,1,...,17.584706,1.914572,0,901838,58.609600,8.76791,7.63069,0.0,1.00000,data\N8.0rv4.0rg8.0Z0.02


## Grouping 

## multiindex data frame test

In [16]:
ag_col_bh['int_type'] = pd.Categorical(
    np.select([
        (ag_col_bh['type1'].isin([0,1]) & ag_col_bh['type2'].isin([0,1])),
        (ag_col_bh['type1'].isin(range(2,10)) & ag_col_bh['type2'].isin(range(2,10))),
        ((ag_col_bh['type1'].isin([0,1]) & ag_col_bh['type2'].isin(range(2,10))) | 
         (ag_col_bh['type1'].isin(range(2,10)) & ag_col_bh['type2'].isin([0,1]))),
        ((ag_col_bh['type1'] >= 10) | (ag_col_bh['type2'] >= 10))
    ], ['MS+MS', 'Giant+Giant', 'MS+Giant', 'Other'], default='Unknown'),
    categories=['MS+MS', 'MS+Giant', 'Giant+Giant', 'Other', 'Unknown'],
    ordered=True
)


In [17]:
df_processed = ag_col_bh.copy()

# Mass ratio categories
df_processed['q'] = pd.cut(
    df_processed['mass_ratio'], 
    bins=[0, 0.01, 0.1, 1.0], 
    labels=['0.01', '0.1', '1'],
    include_lowest=True
)

# Type categories
df_processed['type'] = df_processed['typem'].apply(
    lambda x: 'MS' if x in [0, 1] else 'Giant' if x in [2, 3, 4, 5, 6, 7, 8, 9] else 'Other'
)

# Larger mass categories
df_processed['pm'] = pd.cut(
    df_processed['larger_mass'], 
    bins=[0,10, 50, 100, 150], 
    labels=['10','50', '100', '150'],
    include_lowest=True
)

# Create MultiIndex DataFrame
col = df_processed.set_index(['q', 'type', 'pm','int_type'])

In [22]:
q_list = ['0.01', '0.1', '1']
type_list=['MS','Giant']
pm_list= ['10','50', '100', '150']
int_list=['MS+MS', 'MS+Giant', 'Giant+Giant', 'Other', 'Unknown']
groups = {}
names = []

# # Mass ratio and type m filtering
# for q in q_list:
#     for type in type_list:
#         variable_name = f"col_{q}_{type}"
#         names.append(variable_name)
#         groups[variable_name] = col.loc[q,type,:,:]

# # Mass ratio and interaction type filtering
# for q in q_list:
#     for int in int_list:
#         variable_name = f"col_{q}_{int}"
#         names.append(variable_name)
#         groups[variable_name] = col.loc[q,:,:,int]


# Only interaction type filtering

for int in int_list:
    variable_name = f"col_{int}"
    names.append(variable_name)
    groups[variable_name] = col.loc[:,:,:,int]

        


In [23]:
def sumarise_col(cluster_data):
        col_stats={
        'count': len(cluster_data),
        'percentage': len(cluster_data) / len(col) * 100,
        'avg_smaller_mass': cluster_data['smaller_mass'].mean(),
        'avg_larger_mass': cluster_data['larger_mass'].mean(),
        'avg_mass_ratio': cluster_data['mass_ratio'].mean(),
        'avg_merged_mass': cluster_data['mm'].mean(),
        'median_smaller_mass': cluster_data['smaller_mass'].median(),
        'median_larger_mass': cluster_data['larger_mass'].median(),
        'median_mass_ratio': cluster_data['mass_ratio'].median(),
        'median_merged_mass': cluster_data['mm'].median()

        }

        print("-" * 25) 

        for k, v in col_stats.items():
             print(f"{k:<10} {v:<15}")
        print("-" * 25) 
        return col_stats



In [24]:
group_list=[]
for name in names:

    print(f'Group params {name}')
    sumarise_col(groups[name])
    group_list.append(sumarise_col(groups[name].copy())
)


Group params col_MS+MS
-------------------------
count      3071           
percentage 12.01345694949732
avg_smaller_mass 0.8036139679908825
avg_larger_mass 59.081715971996104
avg_mass_ratio 0.020165477900206452
avg_merged_mass 59.885326310648
median_smaller_mass 0.199938       
median_larger_mass 51.7008        
median_mass_ratio 0.0040506870229007
median_merged_mass 52.2503        
-------------------------
-------------------------
count      3071           
percentage 12.01345694949732
avg_smaller_mass 0.8036139679908825
avg_larger_mass 59.081715971996104
avg_mass_ratio 0.020165477900206452
avg_merged_mass 59.885326310648
median_smaller_mass 0.199938       
median_larger_mass 51.7008        
median_mass_ratio 0.0040506870229007
median_merged_mass 52.2503        
-------------------------
Group params col_MS+Giant
-------------------------
count      22150          
percentage 86.64867190861793
avg_smaller_mass 0.6669368275124153
avg_larger_mass 34.00355099638826
avg_mass_ratio 0.02

In [25]:
group_sum = pd.DataFrame(group_list)
group_sum= group_sum.set_axis(names).round(3)
group_sum

,count,percentage,avg_smaller_mass,avg_larger_mass,avg_mass_ratio,avg_merged_mass,median_smaller_mass,median_larger_mass,median_mass_ratio,median_merged_mass
col_MS+MS,3071,12.013,0.804,59.082,0.020,59.885,0.200,51.701,0.004,52.250
col_MS+Giant,22150,86.649,0.667,34.004,0.023,33.918,0.209,26.104,0.008,26.302
col_Giant+Giant,54,0.211,23.197,33.898,0.749,56.283,19.037,26.193,0.809,44.944
col_Other,3,0.012,1.190,12.853,0.096,14.037,1.182,12.880,0.088,14.014
col_Unknown,285,1.115,1.647,78.878,0.030,74.066,0.196,65.193,0.003,63.329
